# Load Packages

In [24]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.evaluation.consolidate import consolidate_pred_perf, consolidate_ue_perf, consolidate_pi_perf
from src.evaluation.perf_eval import display_pred_perf
from src.evaluation.ue_eval import display_ue_perf, restructure_ue_df
from src.evaluation.pi_eval import display_pi_perf
from src.df_display.latex import df_to_latex, df_to_latex_grouped
from src.misc import create_folder
from ue_pi_dicts import pi_order

seed_list=list(range(2023, 2023+5))
data_label = "mimic"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_evaluation = join(fp_project_checkpoints, "model_evaluation")
fp_consolidated = join(fp_project_checkpoints, "consolidated_results")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prediction Performance

In [5]:
pred_perf_df = consolidate_pred_perf(seed_list, fp_evaluation)
display_pred_perf(pred_perf_df, consolidated=True)
pred_perf_df.to_csv(join(fp_consolidated, "pred_perf.csv"))

,t+1,t+2,t+3
Model,,,
RUE,0.0759 ± 0.001,0.1041 ± 0.001,0.1226 ± 0.001
MC Dropout,0.0764 ± 0.001,0.1051 ± 0.000,0.1235 ± 0.001
GPR,0.0716 ± 0.000,0.0993 ± 0.000,0.1177 ± 0.000
Infer-Noise,0.0759 ± 0.001,0.1041 ± 0.001,0.1226 ± 0.001
BNN,0.078 ± 0.000,0.1059 ± 0.000,0.1246 ± 0.001
DER,0.0762 ± 0.000,0.1051 ± 0.001,0.1217 ± 0.001


In [6]:
print(df_to_latex(pred_perf_df, column_format_dict={"t+1": "min", "t+2": "min", "t+3": "min"}))

\begin{tabular}{cccc}
 & \textbf{t+1} & \textbf{t+2} & \textbf{t+3} \\
Model &  &  &  \\
RUE & \underline{0.0759 ± 0.001} & \underline{0.1041 ± 0.001} & 0.1226 ± 0.001 \\
MC Dropout & 0.0764 ± 0.001 & 0.1051 ± 0.000 & 0.1235 ± 0.001 \\
GPR & \textbf{0.0716 ± 0.000} & \textbf{0.0993 ± 0.000} & \textbf{0.1177 ± 0.000} \\
Infer-Noise & \underline{0.0759 ± 0.001} & \underline{0.1041 ± 0.001} & 0.1226 ± 0.001 \\
BNN & 0.078 ± 0.000 & 0.1059 ± 0.000 & 0.1246 ± 0.001 \\
DER & 0.0762 ± 0.000 & 0.1051 ± 0.001 & \underline{0.1217 ± 0.001} \\
\end{tabular}



# UE Performance

In [7]:
ue_perf_df = consolidate_ue_perf(seed_list, fp_evaluation, exclude_columns="Pval")
display_ue_perf(ue_perf_df, consolidated=True)

t+1


t+2


t+3


In [8]:
print(
    df_to_latex_grouped(
        ue_perf_df, 
        {"Corr": "max", "AURC":"min", "Sigma=0.1": "min", "Sigma=0.2": "min", "Sigma=0.3": "min", "Sigma=0.4": "min"})
)

\begin{tabular}{cccccccc}
\toprule
 &  & Corr & AURC & Sigma=0.1 & Sigma=0.2 & Sigma=0.3 & Sigma=0.4 \\
Time Horizon & Model &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{t+1} & RUE & \textbf{0.431 ± 0.038} & 0.095 ± 0.003 & \underline{0.062 ± 0.007} & 0.079 ± 0.004 & 0.089 ± 0.005 & 0.095 ± 0.006 \\
 & MC Dropout & 0.192 ± 0.028 & 0.125 ± 0.001 & 0.103 ± 0.006 & 0.121 ± 0.002 & 0.129 ± 0.001 & 0.127 ± 0.001 \\
 & GPR & 0.205 ± 0.002 & \underline{0.084 ± 0.000} & 0.07 ± 0.003 & \underline{0.076 ± 0.000} & \underline{0.086 ± 0.001} & \underline{0.092 ± 0.001} \\
 & Infer-Noise & 0.086 ± 0.025 & 0.128 ± 0.002 & 0.124 ± 0.009 & 0.126 ± 0.005 & 0.13 ± 0.005 & 0.129 ± 0.003 \\
 & BNN & \underline{0.285 ± 0.017} & 0.116 ± 0.001 & 0.081 ± 0.004 & 0.104 ± 0.004 & 0.114 ± 0.002 & 0.118 ± 0.001 \\
 & DER & 0.242 ± 0.035 & \textbf{0.074 ± 0.002} & \textbf{0.057 ± 0.003} & \textbf{0.063 ± 0.002} & \textbf{0.071 ± 0.001} & \textbf{0.079 ± 0.001} \\
\cline{1-8}
\multirow[t]{6}{*}{t+2} & RUE & \te

# PI Performance

In [25]:
pi_perf_df = consolidate_pi_perf(
    seed_list, fp_evaluation, selected_columns=["CovP", "PINAW", "PINAFD", "CWFDC"],
    pi_order=pi_order
)
display_pi_perf(pi_perf_df, consolidated=True)
pi_perf_df.to_csv(join(fp_consolidated, "pi_perf.csv"))

t+1:


t+2:


t+3:


In [26]:
print(
    df_to_latex_grouped(
        pi_perf_df, 
        {"PINAW":"min", "PINAFD": "min", "CovP":"min", "CWFDC":"min"})
)

\begin{tabular}{cccccc}
\toprule
 &  & CovP & PINAW & PINAFD & CWFDC \\
Time Horizon & Method &  &  &  &  \\
\midrule
\multirow[t]{10}{*}{t+1} & RUE Gaussian Copula & \textbf{0.00021 ± 0.000} & 0.08563 ± 0.002 & 0.02992 ± 0.001 & \textbf{0.32263 ± 0.018} \\
 & RUE KNN & 0.00029 ± 0.000 & 0.08405 ± 0.001 & 0.03407 ± 0.001 & 0.4075 ± 0.057 \\
 & RUE Conditional Gaussian & 0.00057 ± 0.000 & 0.10275 ± 0.001 & 0.04204 ± 0.001 & 0.7183 ± 0.021 \\
 & RUE Weighted & 0.00105 ± 0.000 & \textbf{0.07228 ± 0.001} & \underline{0.02622 ± 0.001} & 1.15165 ± 0.404 \\
 & RUE Conformal & 0.00043 ± 0.000 & \underline{0.08306 ± 0.004} & 0.02688 ± 0.001 & 0.5443 ± 0.181 \\
 & Infer-Noise Conformal & 0.00023 ± 0.000 & 0.09464 ± 0.001 & 0.03402 ± 0.000 & 0.35519 ± 0.024 \\
 & MC Dropout Conformal & 0.00053 ± 0.000 & 0.0873 ± 0.001 & 0.02885 ± 0.000 & 0.65038 ± 0.036 \\
 & GPR Conformal & \underline{0.00022 ± 0.000} & 0.09093 ± 0.000 & 0.03669 ± 0.000 & \underline{0.34934 ± 0.004} \\
 & BNN Conformal & 0.00045